IMPORTACIÓN DE BIBLIOTECAS


In [1]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import openpyxl
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
import io
import pandas as pd
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
import psycopg2


CONEXIÓN A BASE DE DATOS

In [2]:
def connect_to_db(host, user, password, database, port): # Conexión a la base de datos
    try:
        conexion = psycopg2.connect(
            host=host, 
            user=user,
            password=password,
            database=database,
            port=port
        )
        conexion.autocommit = True # Autoconfirmación
        return conexion
    except psycopg2.Error as e:
        print(f"Error en la conexión a la base de datos: {e}")
        return None

def execute_query(conexion, query): # Ejecución
    try:
        cursor = conexion.cursor()  # Crear un cursor para ejecutar consultas
        cursor.execute(query)
        result = cursor.fetchall()  # Obtener los resultados
        columns = [desc[0] for desc in cursor.description]
        cursor.close()
        return result, columns
    except psycopg2.Error as e:
        print(f"Error en la ejecución de la consulta: {e}")
        return None, None

def close_connection(conexion): # Cierre
    if conexion:
        conexion.close()

FUNCION PARA PROCESAMIENTO DE DATOS

In [3]:
def process_data(result, columns): # Procesamiento de los datos
    df = pd.DataFrame(result, columns=columns) # Crear un DataFrame
    if not df.empty:
        df.dropna(subset=['fecha', 'hora'], inplace=True) # Eliminar filas con valores nulos
        df['Fecha_Hora'] = pd.to_datetime(df['fecha'].astype(str) + ' ' + df['hora'].astype(str), errors='coerce') # Crear una columna con la fecha y hora
        df.drop(columns=['fecha', 'hora'], inplace=True) 
        df.sort_values('Fecha_Hora', inplace=True) # Ordenar por fecha y hora
    return df

def process_precipitation_data(result, columns): # Procesamiento de los datos de precipitación
    df_precip = pd.DataFrame(result, columns=columns) # Crear un DataFrame
    if not df_precip.empty: # Si el DataFrame no está vacío
        df_precip['Fecha_Hora'] = pd.to_datetime(df_precip['fecha'].astype(str) + ' ' + df_precip['hora'].astype(str), errors='coerce') 
        df_precip.drop(columns=['fecha', 'hora'], inplace=True) 
        df_precip.sort_values('Fecha_Hora', inplace=True) 
    return df_precip

GRAFICAR

In [4]:
# ==========================
# CONFIGURACIÓN DE LÍMITES
# ==========================
# Diccionario con los límites del eje Y para gráficos
# Modificar límites de los ejes Y según necesidad
y_limits = {
    
'PA23_03_T': {'y1_min': 878.50, 'y1_max': 882.00},
'PA23_08_T': {'y1_min': 773.00, 'y1_max': 778.40},
'PA23_11_T': {'y1_min': 778.50, 'y1_max': 782.00},
'PA23_12_T': {'y1_min': 842.00, 'y1_max': 848.00},
'PA23_13_T': {'y1_min': 886.00, 'y1_max': 888.0},
'PA23_14_T': {'y1_min': 942.0, 'y1_max': 944.00},
'PA23_17A_T': {'y1_min': 831.00, 'y1_max': 833.00},
'PA23_18_T': {'y1_min': 830.00, 'y1_max': 840.00},
'PA23_20A_T': {'y1_min': 902.00, 'y1_max': 908.00}, 
'PA23_21_T': {'y1_min': 856.00, 'y1_max': 862.00},
'PA23_22_T': {'y1_min': 818.00, 'y1_max': 820.00},
'PA23_27_T': {'y1_min': 728.00, 'y1_max': 730.00},
'PA23_28_T': {'y1_min': 774.00, 'y1_max': 775.00},
'PA23_29_T': {'y1_min': 928.00, 'y1_max': 931.0},
'PA23_30_T': {'y1_min': 1046.2, 'y1_max': 1047.80},
'PA24_01A_T': {'y1_min': 810.00, 'y1_max': 813.00},
'PA24_01_S': {'y1_min': 858.20, 'y1_max': 860.00},
'PA24_01_T': {'y1_min': 910.00, 'y1_max': 918.00},
'PA24_02A_T': {'y1_min': 894.00, 'y1_max': 895.00},
'PA24_02_S': {'y1_min': 870.00, 'y1_max': 872.00},
'PA24_03_S': {'y1_min': 894.00, 'y1_max': 895.50},
'PA24-03-T': {'y1_min': 843.00, 'y1_max': 845.50},
'PA24_03A_T': {'y1_min': 829.00, 'y1_max': 830.00},
'PA24_04_S': {'y1_min': 943.40, 'y1_max': 945.00},
# 'PA24_04A_T': {'y1_min': 825.00, 'y1_max': 832.00},
'PA24_05A_T': {'y1_min': 834.0, 'y1_max': 835.50},
'PA24_05_S': {'y1_min': 934.00, 'y1_max': 935.00},
# 'PA24_05_T': {'y1_min': 804.00, 'y1_max': 810.0},
# 'PA24_06_T': {'y1_min': 802.00, 'y1_max': 815.00},
# 'PA24_09_T': {'y1_min': 905.00, 'y1_max': 907.00},
'PA24_10A_T': {'y1_min': 823.50, 'y1_max': 825.50},
'PA24_10_T': {'y1_min': 797.00, 'y1_max': 799.0},
# 'PA24_07_T': {'y1_min': 802.00, 'y1_max': 807.00},
'PA24-08-T': {'y1_min': 913.0, 'y1_max': 919.0},
'PA24_11_T': {'y1_min': 795.00, 'y1_max': 796.00},
'PA24_12_T': {'y1_min': 883.00, 'y1_max': 885.00},
'PA24_28_T': {'y1_min': 840.00, 'y1_max': 850.00},
# 'PA24_FT_128_T': {'y1_min': 910.00, 'y1_max': 912.50},

}

# ==========================
# FUNCIÓN PARA GRAFICAR
# ==========================
def plot_data(df_instrumento, df_precip, tabla, conexion, excel_path, sheet_name, cell):
    
    # Crear una figura y un eje
    fig, ax1 = plt.subplots(figsize=(14, 7))
    plt.style.use('bmh')  # Estilo de Seaborn

    # Asegurarse de que la columna de 'Fecha_Hora' sea de tipo datetime
    df_instrumento['Fecha_Hora'] = pd.to_datetime(df_instrumento['Fecha_Hora'])

    # ==========================
    # CONFIGURACIÓN DEL EJE Y1 (PRIMARIO)
    # ==========================

    # Configurar los decimales del eje Y primario
    ax1.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))

    # Dibujar la serie de elevación piezométrica
    
    datos_validos = df_instrumento[['Fecha_Hora', 'elevacion_piezometrica']].dropna()
    serie1, = ax1.plot(
        datos_validos['Fecha_Hora'],
        datos_validos['elevacion_piezometrica'],
        color='#00008B',
        label='Elevación Piezométrica (msnm)',
        linewidth=2,
        marker='o',
        markersize=5,
        zorder=3
    )

    # Ajustar los límites del eje Y primario si están definidos
    if tabla in y_limits:
        ax1.set_ylim(y_limits[tabla]['y1_min'], y_limits[tabla]['y1_max'])

    # Calcular minimo y máximo valor de fechas
    fecha_min = df_instrumento['Fecha_Hora'].min()
    fecha_max = df_instrumento['Fecha_Hora'].max()

    # ==========================
    # CONFIGURACIÓN DEL EJE Y2 (SECUNDARIO)
    # ==========================

    # Agrupar y calcular el máximo diario
    df_precip['Fecha_Hora'] = pd.to_datetime(df_precip['Fecha_Hora'])
    df_precip.set_index('Fecha_Hora', inplace=True) # Establecer la columna 'Fecha_Hora' como índice
    df_precip_diario = df_precip.resample('D').max().reset_index() # Agrupar por día y calcular el máximo diario

    # Decidir si usar los datos originales o los datos agrupados por día
    rango_dias = (df_instrumento['Fecha_Hora'].max() - df_instrumento['Fecha_Hora'].min()).days # Calcular el rango de días
    if rango_dias == 0:
        rango_dias = 1  # Evitar división por cero

    # Número de datos de precipitación
    num_datos = len(df_precip) # Número de datos de precipitación
    if num_datos == 0:
        num_datos = 1  # Evitar división por cero      

    # Decidir si usar los datos originales o los datos agrupados por día
    if num_datos / rango_dias > 5000:  # Si la densidad de datos es alta, usar datos diarios
        df_precip_agrupado = df_precip_diario
    else:  # Si la densidad de datos es baja, usar los datos originales
        df_precip_agrupado = df_precip.reset_index()


    # Calcular el ancho de las barras en función de la densidad de los datos
    if fecha_min ==fecha_max:
        ancho_barra = 0.008 # Ancho de las barras si solo hay un punto
    elif (fecha_max - fecha_min).days <= 10:
        ancho_barra = 0.015  # Ancho de las barras si el rango es menor o igual a 10 días
    else:
        ancho_barra = max(0.038, rango_dias / num_datos * 0.15)  # Cálculo dinámico para más días
 
    # Dibujar las barras de precipitación
    if not df_precip_agrupado.empty:
        ax2 = ax1.twinx()
        ax2.bar(
            df_precip_agrupado['Fecha_Hora'], 
            df_precip_agrupado['rain_mm_tot'], 
            label='Precipitación', 
            color='#009ACD', 
            alpha=0.4,          # Transparencia de las barras
            width=ancho_barra,  # Ancho de las barras
            zorder=0            # Dibuja las barras detrás de las líneas
        )
        ax2.set_ylabel('Precipitación (mm/día)', color='black', fontsize=14)
        ax2.tick_params(axis='y', labelcolor='black', labelsize=12)
        ax2.grid(False)  # Desactivar la cuadrícula del eje Y secundario
        ax2.set_ylim(0, df_precip['rain_mm_tot'].max() + 5) # Ajustar el límite superior del eje Y secundario

        # Configurar los decimales del eje Y secundario
        ax2.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.0f'))

    # ==========================
    # CONFIGURACIÓN DEL EJE X
    # ==========================

   
    # Si solo hay un punto, ajusta los límites del eje X para que se centren en ese punto
    if fecha_min == fecha_max:
        fecha_min -= pd.Timedelta(days=1)  # Un día antes
        fecha_max += pd.Timedelta(days=1)  # Un día después
    
    # Definir los límites de fechas en el eje X
    ax1.set_xlim([fecha_min, fecha_max])

    # Calcular el rango de días
    rango_dias = (fecha_max - fecha_min).days
    if rango_dias == 0: # Evitar división por cero
        rango_dias = 1  # Asignar un valor mínimo para evitar errores

    # Ajuste dinámico del intervalo en el eje X
    if rango_dias > 365:
        intervalo = 60  # Cada 2 meses aprox.
    elif rango_dias > 180:
        intervalo = 30  # Cada 1 mes
    elif rango_dias > 90:
        intervalo = 15  # Cada 15 días
    elif rango_dias > 31:
        intervalo = 7  # Cada semana
    elif rango_dias <= 31:
        intervalo = 1  # Cada día
    else:
        intervalo = max(1, rango_dias// 10)  # Dividir en 10 intervalos como máximo

    ax1.xaxis.set_major_locator(mdates.DayLocator(interval=intervalo))  # Intervalo de días
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
    ax1.tick_params(axis='x', labelsize=10, rotation=90)

    # Validar las etiquetas del eje X
    ticks = ax1.get_xticks()
    new_labels = [mdates.num2date(num).strftime('%d-%m-%Y') for num in ticks]
    if len(new_labels) > 0:
        new_labels[-1] = fecha_max.strftime('%d-%m-%Y')  # Cambiar la última etiqueta a la última fecha
    
    # Asegurar que la última fecha esté en las etiquetas del eje X
    # fechas_ticks = list(ax1.get_xticks())  # Obtener los ticks actuales
    # fechas_ticks.append(mdates.date2num(fecha_max))  # Agregar la última fecha como tick
    # ax1.set_xticks(fechas_ticks)  # Actualizar los ticks del eje X
   
   
    # ==========================
    # DISEÑO GENERAL
    # ==========================

    # Cambiar los guiones bajos (_) por guiones (-) en el nombre de la tabla para el título
    titulo_tabla = tabla.replace('_', '-') # Cambiar los guiones bajos (_) por guiones (-) en el nombre de la tabla para el título
    ax1.set_title(f'Nivel Freático - {titulo_tabla}', fontsize=18, fontweight='bold')
    ax1.set_ylabel('Nivel Freático (msnm)', fontsize=14)
    ax1.tick_params(axis='y', labelcolor='black', labelsize=12)
    ax1.tick_params(axis='x', labelsize=12)
    ax1.grid(True, linestyle='--', color='gray', alpha=0.7)

    # Eliminar el borde superior del gráfico
    ax1.spines['top'].set_visible(False)
    if not df_precip.empty:
        ax2.spines['top'].set_visible(False)

    # Eliminar el fondo gris del gráfico
    ax1.set_facecolor('white')

    # Cambiar el color del borde del gráfico
    for spine in ax1.spines.values():
        spine.set_edgecolor('silver')
        spine.set_linewidth(0.01)

    umbrales = {
        'PA24_06_T': {'Nivel Umbral 1': 825.0, 'Nivel Umbral 2': 830.0, 'Nivel Umbral 3': 834.0},
        'PA24_07_T': {'Nivel Umbral 1': 811.0, 'Nivel Umbral 2': 817.0, 'Nivel Umbral 3': 823.0},
        'PA24_04A_T':{'Nivel Umbral 1': 829.0, 'Nivel Umbral 2': 832.0, 'Nivel Umbral 3': 835.0},
        'PA24_05_T':{'Nivel Umbral 1': 810.0, 'Nivel Umbral 2': 815.0, 'Nivel Umbral 3': 820.0},
        'PA24_09_T':{'Nivel Umbral 1': 923.0, 'Nivel Umbral 2': 926.0, 'Nivel Umbral 3': 928.0},
        # 'PA24_08_T':{'Nivel Umbral 1': 930.0, 'Nivel Umbral 2': 938.0, 'Nivel Umbral 3': 945.0},
        # 'PA24_03_T':{'Nivel Umbral 1': 880.0, 'Nivel Umbral 2': 893.0, 'Nivel Umbral 3': 905.0},
        
        # Agrega aquí los instrumentos y sus valores de umbral
    }

    # ==========================
    # LEYENDA
    # ==========================

    # Combinar leyendas de ambos ejes
    lines = [serie1]
    labels = [serie1.get_label()]
    if not df_precip.empty:
        lines.append(Line2D([], [], color='#009ACD', linewidth=4, linestyle='-'))
        labels.append('Precipitación')

    # Agregar líneas de umbral si corresponde
    colores_umbral = ['yellow', 'orange', 'red']
    if tabla in umbrales:
        for i, (nombre_umbral, valor_umbral) in enumerate(umbrales[tabla].items()):
            color = colores_umbral[i] if i < len(colores_umbral) else 'black'
            ax1.axhline(y=valor_umbral, color=color, linestyle='--', linewidth=1.5)
            lines.append(Line2D([0], [0], color=color, linestyle='--', linewidth=1.5))
            labels.append(nombre_umbral)

    # Configuración de la leyenda
    ax1.legend(
        lines, labels, loc='lower center', bbox_to_anchor=(0.5, -0.4), ncol=5, 
        frameon=True, fontsize=12, facecolor='white', edgecolor='silver'
    )

    # Ajustar todo el diseño
    fig.tight_layout(rect=[0, 0, 1, 0.95])

    # Mostrar el gráfico
    plt.show()

    # ==========================
    # GUARDAR EN EXCEL
    # ==========================
    # try:
    #     img_stream = io.BytesIO()
    #     fig.savefig(img_stream, format='png', bbox_inches='tight')
    #     img_stream.seek(0)
    #     img = Image(img_stream)

    #     # Ajustar el tamaño de la imagen
    #     img.width = 17.7 * 37.795275591
    #     img.height = 7 * 37.795275591

    #     # Insertar la imagen en el archivo Excel
    #     wb = load_workbook(excel_path)
    #     ws = wb[sheet_name]

    #     # Buscar y eliminar cualquier imagen existente en la misma celda
    #     for image in ws._images:
    #         if image.anchor._from.col == openpyxl.utils.cell.column_index_from_string(cell[0]) - 1 and \
    #            image.anchor._from.row == int(cell[1:]) - 1:
    #             ws._images.remove(image)
    #             break

    #     ws.add_image(img, cell)
    #     wb.save(excel_path)
    #     print(f"Gráfica insertada en {sheet_name} en la celda {cell}")
    # except Exception as e:
    #     print(f"Error al guardar o insertar la gráfica: {e}")
    # finally:
    #     plt.close(fig)

EJECUTAR CONSULTA Y GRAFICAR

In [5]:
# Asegúrate de ejecutar las celdas anteriores antes de esta celda

# 📅 Parámetros de fecha para filtrar datos de la base de datos
fecha_inicio = '2025-09-01'
fecha_fin = '2025-10-31'


# Diccionario para especificar la hoja y celda donde se insertará cada gráfico
ubicaciones = {

    #=========== SML 2023 ==========#
    "PA23-03-T": ("PA23-03-T", "C20"),   # Nombre de tabla: (Nombre de hoja, Celda)
    "PA23-08-T": ("PA23-08-T", "C20"),
    "PA23-11-T": ("PA23-11-T", "C20"),
    "PA23-12-T": ("PA23-12-T", "C20"),
    "PA23-13-T": ("PA23-13-T", "C20"),
    "PA23-14-T": ("PA23-14-T", "C20"),
    "PA23-17A-T": ("PA23-17A-T", "C20"),
    "PA23-18-T": ("PA23-18-T", "C20"),
    "PA23-20A-T": ("PA23-20A-T", "C20"),
    "PA23-21-T": ("PA23-21-T", "C20"),
    "PA23-22-T": ("PA23-22-T", "C20"),
    "PA23-27-T": ("PA23-27-T", "C20"),
    "PA23-28-T": ("PA23-28-T", "C20"),
    "PA23-29-T": ("PA23-29-T", "C20"),
    "PA23-30-T": ("PA23-30-T", "C20"),

    #=== SML 2024 COMPLEMENTARIOS ====#
    "PA24-01-A-T": ("PA24-01-A-T", "C20"),
    "PA24-01-S": ("PA24-01-S", "C20"),
    "PA24-02-A-T": ("PA24-02-A-T", "C20"),
    "PA24-02-S": ("PA24-02-S", "C20"),
    "PA24-03-A-T": ("PA24-03-A-T", "C20"),
    "PA24-03-S": ("PA24-03-S", "C20"),
    "PA24-04-S": ("PA24-04-S", "C20"),
    "PA24-05-S": ("PA24-05-S", "C20"),
    "PA24-05-A-T": ("PA24-05-A-T", "C20"),
    "PA24-10-A-T": ("PA24-10-A-T", "C20"),
    "PA24-10-T": ("PA24-10-T", "C20"),
    "PA24-11-T": ("PA24-11-T", "C20"),
    "PA24-28-T": ("PA24-28-T", "C20"),
    "PA24-FT-128-T": ("PA24-FT-128-T", "C20"),


    #=== SML 2024 PERFORACIONES ====#
    "PA24-01-T": ("PA24-01-T", "C20"),
    "PA24-02-T": ("PA24-02-T", "C20"),
    "PA24-03-T": ("PA24-03-T", "C20"),
    "PA24-04-A-T": ("PA24-04-A-T", "C20"),
    "PA24-05-T": ("PA24-05-T", "C20"),
    "PA24-06-T": ("PA24-06-T", "C20"),
    "PA24-07-T": ("PA24-07-T", "C20"),
    "PA24-08-T": ("PA24-08-T", "C20"),
    "PA24-09-T": ("PA24-09-T", "C20"),
    "PA24-12-T": ("PA24-12-T", "C20"),


    # Añade más ubicaciones según sea necesario
}

# Ruta del archivo Excel donde se insertarán los gráficos
excel_path = r"\\192.168.60.82\Data$\E1\A4\SCRIPTS\PYTHON\02 GRAFICAS_RDO_MGP\Reporte\2500-DRT-MGP-000-V0.xlsx"

# Conexión a la base de datos y generación de gráficos
conexion = connect_to_db('192.168.60.175', # Host
                         'alexism',  # Usuario
                         'Data.GDR$2024',  # Contraseña
                         'gdr_database',  # Base de datos
                         '5432')      # Puerto

if conexion: 
        query = f'''SELECT "id_instrumento", "fecha", "hora", "elevacion_piezometrica" 
                    FROM "MV_PIEZOMETROS".pz_abiertos
                    WHERE "fecha" BETWEEN '{fecha_inicio}' AND '{fecha_fin}' '''
        result, columns = execute_query(conexion, query)
        df = process_data(result, columns)

        if not df.empty: # Si el DataFrame no está vacío

            # Obtener los instrumentos únicos
            instrumentos_unicos = df['id_instrumento'].unique() # Obtener los instrumentos únicos
            
            # Filtrar los instrumentos a omitir para no graficarlos
            instrumentos_baja = {"PA24_03_A_T", "PA24_02_T"
                 }

            # Filtrar los instrumentos y ordenarlos alfabéticamente       
            tablas = sorted([inst for inst in instrumentos_unicos # Filtrar los instrumentos
                      if inst not in instrumentos_baja]) # Filtrar los instrumentos a omitir
            
            for tabla in tablas:
                # Filtrar los datos por instrumento
                df_instrumento = df[df['id_instrumento'] == tabla].copy() # Filtrar los datos por instrumento
                # Convertir la columna 'Fecha_Hora' a formato datetime
                df_instrumento['Fecha_Hora'] = pd.to_datetime(df_instrumento['Fecha_Hora'])

                if df_instrumento.empty:  # Verificar si el DataFrame está vacío
                    print(f"El instrumento {tabla} no tiene datos para graficar.")
                    continue
                query_precip = f'''SELECT "fecha", "hora", "rain_mm_tot" FROM "MV_NAD_DR"."00_em_via12" 
                                WHERE "fecha" BETWEEN '{fecha_inicio}' AND '{fecha_fin}' '''
                result_precip, columns_precip = execute_query(conexion, query_precip)
                df_precip = process_precipitation_data(result_precip, columns_precip)

                # Obtener la hoja y celda para el gráfico
                if tabla not in ubicaciones:
                    raise KeyError(f"No se encontró una ubicación para el instrumento {tabla}")
                sheet_name, cell = ubicaciones[tabla]
                plot_data(df_instrumento, df_precip, tabla, conexion, excel_path, sheet_name, cell)

close_connection(conexion) # Cerrar la conexión a la base de datos